In [1]:
import numpy as np 
import cv2

STAGE_FIRST_FRAME = 0
STAGE_SECOND_FRAME = 1
STAGE_DEFAULT_FRAME = 2
kMinNumFeature = 1500

lk_params = dict(winSize  = (21, 21), 
                #maxLevel = 3,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

def featureTracking(image_ref, image_cur, px_ref):
    kp2, st, err = cv2.calcOpticalFlowPyrLK(image_ref, image_cur, px_ref, None, **lk_params)  #shape: [k,2] [k,1] [k,1]

    st = st.reshape(st.shape[0])
    kp1 = px_ref[st == 1]
    kp2 = kp2[st == 1]

    return kp1, kp2


class PinholeCamera:
    def __init__(self, width, height, fx, fy, cx, cy, 
                k1=0.0, k2=0.0, p1=0.0, p2=0.0, k3=0.0):
        self.width = width
        self.height = height
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.distortion = (abs(k1) > 0.0000001)
        self.d = [k1, k2, p1, p2, k3]


class VisualOdometry:
    def __init__(self, cam):
        self.frame_stage = 0
        self.cam = cam
        self.new_frame = None
        self.last_frame = None
        self.cur_R = None
        self.cur_t = None
        self.px_ref = None
        self.px_cur = None
        self.focal = cam.fx
        self.pp = (cam.cx, cam.cy)
        self.trueX, self.trueY, self.trueZ = 0, 0, 0
        self.detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)

    def processFirstFrame(self):
        self.px_ref = self.detector.detect(self.new_frame)
        self.px_ref = np.array([x.pt for x in self.px_ref], dtype=np.float32)
        self.frame_stage = STAGE_SECOND_FRAME

    def processSecondFrame(self):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, self.cur_R, self.cur_t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        self.frame_stage = STAGE_DEFAULT_FRAME 
        self.px_ref = self.px_cur

    def processFrame(self, frame_id):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        self.cur_t = self.cur_t + self.cur_R.dot(t) 
        self.cur_R = R.dot(self.cur_R)
        if(self.px_ref.shape[0] < kMinNumFeature):
            self.px_cur = self.detector.detect(self.new_frame)
            self.px_cur = np.array([x.pt for x in self.px_cur], dtype=np.float32)
        self.px_ref = self.px_cur

    def update(self, img, frame_id):
        assert(img.ndim==2 and img.shape[0]==self.cam.height and img.shape[1]==self.cam.width), "Frame: provided image has not the same size as the camera model or image is not grayscale"
        self.new_frame = img
        if(self.frame_stage == STAGE_DEFAULT_FRAME):
            self.processFrame(frame_id)
        elif(self.frame_stage == STAGE_SECOND_FRAME):
            self.processSecondFrame()
        elif(self.frame_stage == STAGE_FIRST_FRAME):
            self.processFirstFrame()
        self.last_frame = self.new_frame

In [2]:
cam = PinholeCamera(1242.0, 375.0, 718.8560, 718.8560, 607.1928, 185.2157)
vo = VisualOdometry(cam)

traj = np.zeros((600,600,3), dtype=np.uint8)

for img_id in range(76):
    img = cv2.imread('data/'+str(img_id).zfill(10)+'.png', 0)

    vo.update(img, img_id)

    cur_t = vo.cur_t
    if(img_id > 2):
        x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else:
        x, y, z = 0., 0., 0.
    draw_x, draw_y = int(x)+290, int(z)+90
    true_x, true_y = int(vo.trueX)+290, int(vo.trueZ)+90

    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/4540,255-img_id*255/4540,0), 1)
    cv2.circle(traj, (true_x,true_y), 1, (0,0,255), 2)
    cv2.rectangle(traj, (10, 20), (600, 60), (0,0,0), -1)
    text = "Coordinates: x=%2fm y=%2fm z=%2fm"%(x,y,z)
    cv2.putText(traj, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)

    cv2.imshow('Road facing camera', img)
    cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)

cv2.imwrite('map.png', traj)

[[ 0.99998821  0.00298161  0.00383225]
 [-0.00299705  0.99998738  0.00403155]
 [-0.00382019 -0.00404298  0.99998453]] [[-0.00161762]
 [ 0.0206138 ]
 [ 0.9997862 ]]
[[ 0.99998953  0.00410675  0.0020208 ]
 [-0.00411857  0.99997424  0.00587796]
 [-0.00199661 -0.00588623  0.99998068]] [[ 0.00906588]
 [-0.02164996]
 [ 0.99972451]]
[[ 0.99999574 -0.00164352  0.00241233]
 [ 0.00163935  0.99999716  0.00172831]
 [-0.00241516 -0.00172435  0.9999956 ]] [[ 0.01648664]
 [-0.01922885]
 [ 0.99967917]]
[[ 9.99996537e-01 -2.63144415e-03  3.75865040e-05]
 [ 2.63161498e-03  9.99980459e-01 -5.67071298e-03]
 [-2.26636050e-05  5.67079226e-03  9.99983921e-01]] [[ 0.01109053]
 [-0.02137243]
 [ 0.99971007]]
[[ 0.99997477  0.00675354 -0.0022037 ]
 [-0.00676146  0.99997063 -0.00360794]
 [ 0.00217927  0.00362275  0.99999106]] [[-0.01208589]
 [-0.00680255]
 [ 0.99990382]]
[[ 0.99999699  0.00104651 -0.00221703]
 [-0.00105253  0.99999576 -0.0027169 ]
 [ 0.00221418  0.00271923  0.99999385]] [[0.00756929]
 [0.01192366

[[ 9.99995374e-01 -8.43423999e-04 -2.92243127e-03]
 [ 8.48105939e-04  9.99998358e-01  1.60120155e-03]
 [ 2.92107598e-03 -1.60367267e-03  9.99994448e-01]] [[-0.01710968]
 [-0.0098985 ]
 [ 0.99980462]]
[[ 9.99993728e-01 -3.16237422e-04 -3.52772864e-03]
 [ 3.16476664e-04  9.99999948e-01  6.72595803e-05]
 [ 3.52770718e-03 -6.83756022e-05  9.99993775e-01]] [[-0.00830266]
 [-0.01248811]
 [ 0.99988755]]
[[ 9.99995712e-01 -8.31015154e-04 -2.80811489e-03]
 [ 8.21515809e-04  9.99993943e-01 -3.38228112e-03]
 [ 2.81090861e-03  3.37995971e-03  9.99990337e-01]] [[-0.0120192 ]
 [-0.01034225]
 [ 0.99987428]]
[[ 0.99999624 -0.00158128 -0.00224104]
 [ 0.00157806  0.99999772 -0.00143735]
 [ 0.00224331  0.00143381  0.99999646]] [[-0.00962672]
 [-0.01757201]
 [ 0.99979926]]
[[ 9.99997939e-01  8.30271355e-04 -1.85293106e-03]
 [-8.27236701e-04  9.99998316e-01  1.63792457e-03]
 [ 1.85428787e-03 -1.63638838e-03  9.99996942e-01]] [[-0.01644205]
 [-0.00373589]
 [ 0.99985784]]
[[ 0.99999798  0.00116071 -0.0016428

True